In [1]:
# 실행 후 런타임 다시시작
!pip install -q kfp==0.2.5 --upgrade --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 4.9.2 requires ruamel_yaml>=0.11.14, which is not installed.
tfx 0.26.3 requires docker<5,>=4.1, but you have docker 5.0.0 which is incompatible.
tfx 0.26.3 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.1.0 which is incompatible.
tfx 0.26.3 requires kubernetes<12,>=10.0.1, but you have kubernetes 10.0.0 which is incompatible.
tfx 0.26.3 requires pyarrow<0.18,>=0.17, but you have pyarrow 3.0.0 which is incompatible.
tensorflow-pr

In [1]:
import kfp
from kfp import dsl

def add_two_numbers(a, b):
    # 두 개의 값을 인자로 받아서 더하는 함수
    return dsl.ContainerOp(
        name='calculate_sum',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/results.txt", "a") as file: file.write(str({} + {}))'.format(a, b)],
        file_outputs={
            'data': '/tmp/results.txt',
        }
    )

def echo_op(text):
    # 결과 값을 출력하는 함수
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "Result: {}"'.format(text)]
    )

In [2]:
@dsl.pipeline(
  name='Calcualte sum pipeline',
  description='Calculate sum of numbers and prints the result.'
)
def calculate_sum(a=7, b=10, c=4, d=7):
    # 처음 두 개를 병렬 처리하도록 파이프라인을 구성
    
    sum1 = add_two_numbers(a, b)
    sum2 = add_two_numbers(c, d)
    sum = add_two_numbers(sum1.output, sum2.output)

    echo_task = echo_op(sum.output)

In [3]:
# pipeline 컴파일
kfp.compiler.Compiler().compile(calculate_sum, 'calculate-sum-pipeline.zip')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "7".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/home/jupyter/.local/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "10".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/home/jupyter/.local/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "4".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [4]:
# 파이프라인 배포
client = kfp.Client(host = '7ecd74b1b09e1376-dot-us-central1.pipelines.googleusercontent.com') # 본인의 host wn
my_experiment = client.create_experiment(name='my_experiment')
my_run = client.run_pipeline(my_experiment.id, 'calculate-sum-pipeline', 
  'calculate-sum-pipeline.zip')